In [ ]:
import pandas as pd

# Define the path to the Excel file and the output CSV file
xlsx_file = '/content/drive/MyDrive/sentimental_analysis/webscrap_Hindi.xlsx'  # Replace with your Excel file path
csv_file = '/content/drive/MyDrive/sentimental_analysis/webscrap_Hindi.csv'   # Replace with your desired CSV file path

# Read the Excel file
df = pd.read_excel(xlsx_file)

# Convert the DataFrame to a CSV file
df.to_csv(csv_file, index=False)

print(f"Conversion complete: '{xlsx_file}' has been converted to '{csv_file}'")

Conversion complete: '/content/drive/MyDrive/sentimental_analysis/webscrap_Hindi.xlsx' has been converted to '/content/drive/MyDrive/sentimental_analysis/webscrap_Hindi.csv'


In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN

In [ ]:
import pandas as pd
import torch
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
import torch.nn.functional as F
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, Trainer, TrainingArguments

model_name = 'xlm-roberta-base'
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
model = XLMRobertaForSequenceClassification.from_pretrained(model_name, num_labels=3)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = sentiment_model(**inputs)
    probs = F.softmax(outputs.logits, dim=-1)

    pos_score = probs[0][2].item()  # Assuming index 2 is POSITIVE
    neu_score = probs[0][1].item()  # Assuming index 1 is NEUTRAL
    neg_score = probs[0][0].item()  # Assuming index 0 is NEGATIVE

    return pos_score, neu_score, neg_score

def calculate_intensity(pos_score, neu_score, neg_score):
    intensity = pos_score - neg_score  # Intensity between -1 and 1

    # Normalize intensity to be between -1 and 1
    if intensity > 0:
        intensity = min(intensity, 1)
    else:
        intensity = max(intensity, -1)

    return round(intensity, 2)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/sentimental_analysis/webscrap_Hindi.csv')
texts = df['Title'].astype(str).fillna('')

In [ ]:
def preprocess_text(text):
    return text.lower()

In [ ]:
df['Title'] = df['Title'].apply(preprocess_text)

In [ ]:
def get_sentiment(texts, model, tokenizer):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=-1)
        predictions = torch.argmax(probs, dim=-1)
    return predictions, probs


In [ ]:
df.to_csv('/content/drive/MyDrive/sentimental_analysis/output_sentiment_Hindi.csv', index=False)
print("Sentiment and intensity analysis completed and results saved to 'output_with_sentiments_and_intensity.csv'")

Sentiment and intensity analysis completed and results saved to 'output_with_sentiments_and_intensity.csv'


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the tokenizer and model
model_name = "mrm8488/bert-tiny-finetuned-fake-news-detection"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
def predict_fake_news(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # Perform forward pass with the model
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the logits and convert them to probabilities
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1).squeeze().tolist()

    # Extracting the class with the highest probability
    predicted_class = torch.argmax(logits, dim=1).item()

    labels = ["Real", "Fake"]
    return labels[predicted_class], probabilities

In [ ]:
# Create empty lists to hold the results
predictions = []
real_probs = []
fake_probs = []

# Loop through each description and predict
for description in df['Title']:
    prediction, probs = predict_fake_news(description)
    predictions.append(prediction)
    real_probs.append(probs[0])
    fake_probs.append(probs[1])

# Add the results to the DataFrame
df['Prediction'] = predictions
df['Real_Prob'] = real_probs
df['Fake_Prob'] = fake_probs

In [ ]:
# Save the DataFrame with predictions to a new CSV file
df.to_csv("/content/drive/MyDrive/sentimental_analysis/output_fake_or_real_Hindi.csv", index=False)

# Display the first few rows of the updated DataFrame
print(df.head())

   Index                                              Title  Description  \
0      0  भास्कर एक्सक्लूसिवकोलकाता रेप-मर्डर केस…आरोपी ...          NaN   
1      1  भास्कर एक्सक्लूसिवकोलकाता रेप-मर्डर केस…आरोपी ...          NaN   
2      2  केदारनाथ में एयरलिफ्ट किया जा रहा हेलिकॉप्टर ग...          NaN   
3      3  केदारनाथ में एयरलिफ्ट किया जा रहा हेलिकॉप्टर ग...          NaN   
4      4  भास्कर एक्सप्लेनर- मौलवी के पास शादी का रजिस्ट...          NaN   

   Area of News  positive   neutral  negative  intensity Prediction  \
0           NaN  0.225524  0.680865  0.093611       0.13       Real   
1           NaN  0.225524  0.680865  0.093611       0.13       Real   
2           NaN  0.225524  0.680865  0.093611       0.13       Real   
3           NaN  0.225524  0.680865  0.093611       0.13       Real   
4           NaN  0.225524  0.680865  0.093611       0.13       Real   

   Real_Prob  Fake_Prob  
0   0.989388   0.010612  
1   0.989388   0.010612  
2   0.963769   0.036231  
3   0.963769